In [47]:
import spacy
nlp = spacy.load('en_core_web_sm') 

from sutime import SUTime
sutime = SUTime(mark_time_ranges=True, include_range=True)

import csv, sqlite3

In [48]:
# variables
query_file_path = "../possible-questions.txt"

additional_stopwords=['case', 'find', 'covid', 'coronavirus', 'covid-19', 'covid19', 'world']

assign_base_words={
    'recover' : ['recover','recovery','cure','heal'],
    'death' : ['death','fatality','fatal','demise','decease','die','expire'],
    'confirm': ['confirm'],
    'active' : ['active', 'live'],
    'maximum' : ['maximum', 'high', 'max', 'maximal', 'most'],
    'minimum' : ['minimum', 'low', 'least', 'min'],
    'average' : ['average', 'avg', 'normally', 'usually', 'generally'],
    'state' : ['state', 'province'],
    'country' : ['country', 'region', 'nation', 'place']
}

In [49]:
def read_queries(query_file_path):
    queries=[]
    with open(query_file_path,"r") as f:
        for line in f.readlines():
            line = line[:-1]
            if(line):
                queries.append(line)

    return queries
    
queries = read_queries(query_file_path)

In [50]:
def print_entities(sentence):
	doc = nlp(sentence)
	print("----> Entities:")
	for ent in doc.ents: 
		print("-------->",ent.text, ent.start_char, ent.end_char, ent.label_)

def print_tokens(sentence):
	doc = nlp(sentence) 
	print("----> Tokens:")
	for token in doc: 
		print("-------->", token.text, token.pos_, token.dep_) 

In [51]:
def add_stopwords(additional_stopwords):
    for word in additional_stopwords:
        nlp.vocab[word].is_stop=True

def get_reverse_dict(assign_base_words):
    reverse_base_word_dict={}
    for base, l in assign_base_words.items():
        for item in l:
            doc = nlp(item)
            item = doc[0].lemma_
            reverse_base_word_dict[item]=base

    return reverse_base_word_dict

# table = nlp.vocab.lookups.get_table("lemma_lookup")
# for base, l in assign_base_words.items():
#     for item in l:
#         table[item]=base

# doc=nlp("recovery")
# print(doc[0].lemma_)

add_stopwords(additional_stopwords)
reverse_base_word_dict = get_reverse_dict(assign_base_words)
print(reverse_base_word_dict)

{'recover': 'recover', 'recovery': 'recover', 'cure': 'recover', 'heal': 'recover', 'death': 'death', 'fatality': 'death', 'fatal': 'death', 'demise': 'death', 'decease': 'death', 'die': 'death', 'expire': 'death', 'confirm': 'confirm', 'active': 'active', 'live': 'active', 'maximum': 'maximum', 'high': 'maximum', 'max': 'maximum', 'maximal': 'maximum', 'most': 'maximum', 'minimum': 'minimum', 'low': 'minimum', 'least': 'minimum', 'min': 'minimum', 'average': 'average', 'avg': 'average', 'normally': 'average', 'usually': 'average', 'generally': 'average', 'state': 'state', 'province': 'state', 'country': 'country', 'region': 'country', 'nation': 'country', 'place': 'country'}


In [52]:
def get_places(entities):
    places=[]
    for ent in entities:
        if ent.label_ == 'GPE':
            places.append(ent.text)
    
    if len(places) == 0:
        places.append('world')
    
    return places

def get_time_duration(query):
    time={}
    parsed=sutime.parse(query)

    if len(parsed)==1:
        item=parsed[0]

        if item['type']=='DATE':
            if item['value'] != 'PRESENT_REF':
                time['begin']=item['value']
                time['end']=item['value']

        elif item['type']=='DURATION' and type(item['value'])==type(time):
            time['begin']=item['value']['begin']
            time['end']=item['value']['end']
    
    elif len(parsed)==2 and parsed[0]['type']=='DATE' and parsed[1]['type']=='DATE':
        time['begin']=min(parsed[0]['value'], parsed[1]['value'])
        time['end']=max(parsed[0]['value'], parsed[1]['value'])

    # print(parsed)
    # for item in parsed:
    #     time.append(item['value'])

    if len(time)==0:
        time['begin']='XXXX-XX-XX'
        time['end']='XXXX-XX-XX'
    
    time['begin']='2020'+time['begin'][4:]
    time['end']='2020'+time['end'][4:]

    return time

def remove_unnecessary(query):
    doc=nlp(query)

    for ent in doc.ents:
        if ent.label_=='GPE' or ent.label_=='DATE':
            query=query.replace(ent.text,"")
    # print(query)

    query=query.lower()
    doc=nlp(query)
    query=' '.join([token.lemma_ for token in doc])

    for word,base in reverse_base_word_dict.items():
        query=query.replace(word,base)

    doc=nlp(query)
    for token in doc:
        if token.is_stop==True or token.dep_=='prep' or token.dep_=='punct':
            query=query.replace(token.text,"")
    
    return query

def get_case_and_function_type(query):
    processed_query = remove_unnecessary(query)
    # print(processed_query)

    case_types = ['confirm','recover','death','increase rate']
    function_types = ['maximum', 'minimum', 'average', 'sum']

    final_case=""
    for case in case_types:
        if(processed_query.find(case)>=0):
            final_case = case
            break
    
    if final_case=="":
        final_case='confirm'

    final_func=""
    for func in function_types:
        if(processed_query.find(func)>=0):
            final_func = func
            break
    
    if final_func=="":
        final_func='sum'
    
    return final_case, final_func

def get_operation_type(query):
    operation_type = ['state', 'country']

    query=query.lower()
    doc=nlp(query)
    query=' '.join([token.lemma_ for token in doc])

    for word,base in reverse_base_word_dict.items():
        query=query.replace(word,base)
    
    doc=nlp(query)
    for token in doc:
        if token.dep_!='compound':
            if token.text == 'state' or token.text == 'country':
                return token.text

    return 'cases'

def parse_parameters(query):
    doc=nlp(query)
    entities=doc.ents

    place = get_places(entities)
    time_duration = get_time_duration(query)
    case_type, function_type = get_case_and_function_type(query)
    operation_type = get_operation_type(query)

    print("Place -> ",place)
    print("Time Duration -> ", time_duration)
    print('Case Type -> ', case_type)
    print('Function Type -> ', function_type)
    print('Operation Type -> ', operation_type)

In [53]:
def process_query(query):
    print(query)
    parse_parameters(query)
    # print_entities(query)
    #print_tokens(sent)
    # doc=nlp(query)
    # print([chunk.text for chunk in doc.noun_chunks])
    print()

for q in queries:
    process_query(q)

Recovery cases in Bombay, India?
Place ->  ['Bombay', 'India']
Time Duration ->  {'begin': '2020-XX-XX', 'end': '2020-XX-XX'}
Case Type ->  recover
Function Type ->  sum
Operation Type ->  cases

which country has the highest number of cases in a single state?
Place ->  ['world']
Time Duration ->  {'begin': '2020-XX-XX', 'end': '2020-XX-XX'}
Case Type ->  confirm
Function Type ->  sum
Operation Type ->  country

which place has the highest number of cases in US?
Place ->  ['US']
Time Duration ->  {'begin': '2020-XX-XX', 'end': '2020-XX-XX'}
Case Type ->  confirm
Function Type ->  maximum
Operation Type ->  country

state having maximum number of cases?
Place ->  ['world']
Time Duration ->  {'begin': '2020-XX-XX', 'end': '2020-XX-XX'}
Case Type ->  confirm
Function Type ->  maximum
Operation Type ->  state

maximum number of cases in state Alabama?
Place ->  ['Alabama']
Time Duration ->  {'begin': '2020-XX-XX', 'end': '2020-XX-XX'}
Case Type ->  confirm
Function Type ->  maximum
Operati

In [2]:
def csv_to_table(path, csv_name, table_name):
    with open(dataset_path+csv_name,'r') as fin:
        dr = csv.DictReader(fin)
        to_db=[tuple(i.values()) for i in dr]
    
    count=len(dr.fieldnames)
    bindings="?, "*count

    cur.executemany("INSERT INTO "+table_name+" VALUES ("+bindings[:-2]+");", to_db)

In [3]:
# con = sqlite3.connect(r"..\dataset\covid-19\mysql_database\covid19.db")
# cur = con.cursor()

# dataset_path="../dataset/covid-19/required_only/"

# tables=[("worldwide_aggregate"),("reference"),("timeseries"),("us")]
# for table in tables:
#     cur.execute("DROP TABLE IF EXISTS "+table+";")

# cur.execute("create table worldwide_aggregate(Date Date NOT NULL, Confirmed BIGINT NOT NULL, Recovered BIGINT NOT NULL, Deaths BIGINT NOT NULL, Increase_rate FLOAT default NULL, PRIMARY KEY (Date));")

# csv_to_table(dataset_path,"worldwide-aggregate.csv", tables[0])

# cur.execute("create table us(Date Date NOT NULL, Admin2 VARCHAR(100) NOT NULL, Province_State VARCHAR(100) NOT NULL, Confirmed BIGINT NOT NULL, Deaths BIGINT NOT NULL, Country_Region VARCHAR(100) NOT NULL, PRIMARY KEY (Date, Admin2, Province_State));")

# csv_to_table(dataset_path,"us_simplified.csv", tables[3])

# cur.execute("create table reference(UID INT NOT NULL, iso2 VARCHAR(20), iso3 VARCHAR(20), code3 INT, FIPS INT, Admin2 VARCHAR(100) NOT NULL, Province_State VARCHAR(100) NOT NULL, Country_Region VARCHAR(100) NOT NULL, Lat FLOAT NOT NULL, Long_ FLOAT NOT NULL, Combined_Key VARCHAR(100), Popolation BIGINT NOT NULL, PRIMARY KEY (UID));")

# csv_to_table(dataset_path,"reference.csv", tables[1])

# cur.execute("create table timeseries(Date Date NOT NULL, Country_Region VARCHAR(100) NOT NULL, Province_State VARCHAR(100), Confirmed BIGINT NOT NULL, Recovered BIGINT NOT NULL, Deaths BIGINT NOT NULL, PRIMARY KEY (Date, Country_Region, Province_State));")

# csv_to_table(dataset_path,"time-series-19-covid-combined.csv", tables[2])

# con.commit()

# # with open('../dataset/covid-19/mysql_database/dump.sql','w') as fp:
# #     for line in con.iterdump():
# #         fp.write('%s\n' % line)

# con.close()
